In [80]:
import os, sys
sys.path.append(os.path.abspath(os.path.join('../', 'wick-main')))

In [81]:
from fractions import Fraction

from wick.expression import AExpression
from wick.wick import apply_wick
from wick.convenience import commute
from wick.convenience import one_e, two_e, E0, E1, E2, braE1

In [82]:
def hf_energy_equation(H):
    out = apply_wick(H)
    out.resolve()

    final = AExpression(Ex=out, simplify=False)

    nterm = len(final.terms)
    
    final.simplify()
    for term in final.terms:
        for t in term.tensors:
            # if a bracket is found remove all 
            # the characters in the bracket
            if "(" in t.name:
                t.name = t.name.split("(")[0]

    print("\nEHF (After simplify): %d -> %d" % (nterm, len(final.terms)))
    print(final)

H1 = one_e("f", ["occ", "vir"], norder=False)
H2 = two_e("I", ["occ", "vir"], norder=False, anti=False)
hf_energy_equation(H1 + H2)


EHF (After simplify): 3 -> 3
 + 1.0\sum_{i}f_{ii}
 + 0.5\sum_{ij}I_{ijij}
 - 0.5\sum_{ij}I_{ijji}


In [83]:
def ccsd_t1_equation(H):
    T1 = E1("t", ["occ"], ["vir"])
    T2 = E2("t", ["occ"], ["vir"])
    T = T1 + T2

    HT = commute(H, T)
    HTT = commute(HT, T)
    bra = braE1("occ", "vir")
    S = bra * (H + HT + HTT  * Fraction(1, 2))

    out = apply_wick(S)
    out.resolve()

    final = AExpression(Ex=out, simplify=False)

    nterm = len(final.terms)
    
    final.simplify()
    for term in final.terms:
        for t in term.tensors:
            # if a bracket is found remove all 
            # the characters in the bracket
            if "(" in t.name:
                t.name = t.name.split("(")[0]

    print("\nR1 (After simplify): %d -> %d" % (nterm, len(final.terms)))
    print(final)

H1 = one_e("f", ["occ", "vir"], norder=True)
H2 = two_e("I", ["occ", "vir"], norder=True, anti=False)
ccsd_t1_equation(H1 + H2)


R1 (After simplify): 129 -> 17
 + 1.0f_{ai}
 - 1.0\sum_{j}f_{ji}t_{aj}
 + 1.0\sum_{b}f_{ab}t_{bi}
 + 1.0\sum_{jb}f_{jb}t_{abij}
 - 1.0\sum_{jb}I_{jaib}t_{bj}
 + 1.0\sum_{jb}I_{jabi}t_{bj}
 - 1.0\sum_{jkb}I_{jkib}t_{abjk}
 - 1.0\sum_{jbc}I_{jabc}t_{bcij}
 - 1.0\sum_{jb}f_{jb}t_{aj}t_{bi}
 - 1.0\sum_{jkb}I_{jkib}t_{aj}t_{bk}
 + 1.0\sum_{jkb}I_{jkib}t_{ak}t_{bj}
 - 1.0\sum_{jbc}I_{jabc}t_{bi}t_{cj}
 + 1.0\sum_{jbc}I_{jabc}t_{bj}t_{ci}
 - 1.0\sum_{jkbc}I_{jkbc}t_{aj}t_{bcik}
 - 1.0\sum_{jkbc}I_{jkbc}t_{bi}t_{acjk}
 + 1.0\sum_{jkbc}I_{jkbc}t_{bj}t_{acik}
 - 1.0\sum_{jkbc}I_{jkbc}t_{bk}t_{acij}


In [84]:
import wick

from wick.index import Idx
from wick.operator import Sigma, TensorSym, FOperator, Tensor, normal_ordered
from wick.expression import Term, Expression

from itertools import product
from fractions import Fraction

def PhysNotaERIs(name, spaces, norder=False):
    terms = []
    sym = TensorSym([(0, 1, 2, 3), (1, 0, 3, 2)], [1, 1])

    for ss in product(spaces, repeat=4):
        # Count how many times each space appears
        inds = [Idx(sum(s == si for s in ss[:i]), si) for i, si in enumerate(ss)]
        p, q, r, s = inds
        fops = [FOperator(p, 1), FOperator(q, 1), FOperator(s, 0), FOperator(r, 0)]

        sign = 1
        if norder:
            fops, sign = normal_ordered(fops)

        term = Term(
            sign * Fraction(1, 2),         # The scalar factor
            list(map(Sigma, inds)), # The summed indices
            [Tensor(inds, name, sym=sym)], # The tensor
            fops, [] # The operators and the bra.
        )

        terms.append(term)

    return Expression(terms)

H1 = one_e("f", ["occ", "vir"], norder=False)
H2 = PhysNotaERIs("I", ["occ", "vir"], norder=False)
hf_energy_equation(H1 + H2)


EHF (After simplify): 3 -> 3
 + 1.0\sum_{i}f_{ii}
 + 0.5\sum_{ij}I_{ijij}
 - 0.5\sum_{ij}I_{ijji}


In [85]:
H1 = one_e("f", ["occ", "vir"], norder=True)
H2 = PhysNotaERIs("I", ["occ", "vir"], norder=True)
ccsd_t1_equation(H1 + H2)


R1 (After simplify): 129 -> 17
 + 1.0f_{ai}
 - 1.0\sum_{j}f_{ji}t_{aj}
 + 1.0\sum_{b}f_{ab}t_{bi}
 + 1.0\sum_{jb}f_{jb}t_{abij}
 - 1.0\sum_{jb}I_{jaib}t_{bj}
 + 1.0\sum_{jb}I_{jabi}t_{bj}
 - 1.0\sum_{jkb}I_{jkib}t_{abjk}
 - 1.0\sum_{jbc}I_{jabc}t_{bcij}
 - 1.0\sum_{jb}f_{jb}t_{aj}t_{bi}
 - 1.0\sum_{jkb}I_{jkib}t_{aj}t_{bk}
 + 1.0\sum_{jkb}I_{jkib}t_{ak}t_{bj}
 - 1.0\sum_{jbc}I_{jabc}t_{bi}t_{cj}
 + 1.0\sum_{jbc}I_{jabc}t_{bj}t_{ci}
 - 1.0\sum_{jkbc}I_{jkbc}t_{aj}t_{bcik}
 - 1.0\sum_{jkbc}I_{jkbc}t_{bi}t_{acjk}
 + 1.0\sum_{jkbc}I_{jkbc}t_{bj}t_{acik}
 - 1.0\sum_{jkbc}I_{jkbc}t_{bk}t_{acij}


In [86]:
def ChemNotaERIs(name, spaces, norder=False):
    terms = []
    sym = TensorSym([(0, 1, 2, 3), (0, 1, 3, 2), (1, 0, 2, 3), (1, 0, 3, 2)], [1, 1, 1, 1])

    for ss in product(spaces, repeat=4):
        # Count how many times each space appears
        inds = [Idx(sum(s == si for s in ss[:i]), si) for i, si in enumerate(ss)]
        p, q, r, s = inds
        fops = [FOperator(p, 1), FOperator(r, 1), FOperator(s, 0), FOperator(q, 0)]

        sign = 1
        if norder:
            fops, sign = normal_ordered(fops)

        terms.append(
            Term(
                sign * Fraction(1, 2),  # The scalar factor
                list(map(Sigma, inds)), # The summed indices
                [Tensor(inds, name, sym=sym)], # The tensor
                fops, [] # The operators and the bra.
            )
        )

    return Expression(terms)

H1 = one_e("f", ["occ", "vir"], norder=False)
H2 = ChemNotaERIs("I", ["occ", "vir"], norder=False)
hf_energy_equation(H1 + H2)


EHF (After simplify): 3 -> 3
 + 1.0\sum_{i}f_{ii}
 + 0.5\sum_{ij}I_{iijj}
 - 0.5\sum_{ij}I_{ijji}


In [87]:
H1 = one_e("f", ["occ", "vir"], norder=True)
H2 = ChemNotaERIs("I", ["occ", "vir"], norder=True)
ccsd_t1_equation(H1 + H2)


R1 (After simplify): 129 -> 29
 + 1.0f_{ai}
 - 1.0\sum_{j}f_{ji}t_{aj}
 + 1.0\sum_{b}f_{ab}t_{bi}
 + 1.0\sum_{jb}f_{jb}t_{abij}
 - 0.5\sum_{jb}I_{jiab}t_{bj}
 + 0.5\sum_{jb}I_{jbai}t_{bj}
 + 0.5\sum_{jb}I_{aijb}t_{bj}
 - 0.5\sum_{bj}I_{abji}t_{bj}
 - 0.5\sum_{jkb}I_{jikb}t_{abjk}
 + 0.5\sum_{jbk}I_{jbki}t_{abjk}
 - 0.5\sum_{jbc}I_{jbac}t_{bcij}
 + 0.5\sum_{bjc}I_{abjc}t_{bcij}
 - 1.0\sum_{jb}f_{jb}t_{aj}t_{bi}
 - 0.5\sum_{jkb}I_{jikb}t_{aj}t_{bk}
 + 0.5\sum_{jkb}I_{jikb}t_{ak}t_{bj}
 + 0.5\sum_{jbk}I_{jbki}t_{aj}t_{bk}
 - 0.5\sum_{jbk}I_{jbki}t_{ak}t_{bj}
 - 0.5\sum_{jbc}I_{jbac}t_{bi}t_{cj}
 + 0.5\sum_{jbc}I_{jbac}t_{bj}t_{ci}
 + 0.5\sum_{bjc}I_{abjc}t_{bi}t_{cj}
 - 0.5\sum_{bjc}I_{abjc}t_{bj}t_{ci}
 - 0.5\sum_{jbkc}I_{jbkc}t_{aj}t_{bcik}
 + 0.5\sum_{jbkc}I_{jbkc}t_{ak}t_{bcij}
 - 0.5\sum_{jbkc}I_{jbkc}t_{bi}t_{acjk}
 + 0.5\sum_{jbkc}I_{jbkc}t_{bj}t_{acik}
 - 0.5\sum_{jbkc}I_{jbkc}t_{bk}t_{acij}
 + 0.5\sum_{jbkc}I_{jbkc}t_{ci}t_{abjk}
 - 0.5\sum_{jbkc}I_{jbkc}t_{cj}t_{abik}
 + 0.5\su

In [88]:
from wick.operator import  Delta
from wick.expression import ATerm

from wick.index import Idx
from wick.operator import Sigma, TensorSym, FOperator, Tensor, normal_ordered
from wick.expression import Term, Expression

from itertools import product
from fractions import Fraction

def DensityFittingERI(name, spaces, norder=False):
    terms = []
    sym = TensorSym([(0, 1, 2), (0, 2, 1)], [1, 1])

    for ss in product(spaces, repeat=4):
        # Count how many times each space appears
        inds = [Idx(sum(s == si for s in ss[:i]), si) for i, si in enumerate(ss)]
        p, q, r, s = inds
        P = Idx(0, "aux")
        fops = [FOperator(p, 1), FOperator(r, 1), FOperator(s, 0), FOperator(q, 0)]

        sign = 1
        if norder:
            fops, sign = normal_ordered(fops)

        t = Term(0.5 * sign,  # The scalar factor
                [Sigma(P), Sigma(p), Sigma(q), Sigma(r), Sigma(s)],
                [Tensor([P, p, q], name + "(0)", sym=sym), Tensor([P, r, s], name + "(1)", sym=sym)], # The tensor
                fops, []
            )
        t.index_key = {
            "occ": "ijklmno",
            "vir": "abcdefg",
            "aux": "PQRSIJKL"
            }

        terms.append(t)

    return Expression(terms)

H1 = one_e("f", ["occ", "vir"], norder=False)
H2 = DensityFittingERI("L", ["occ", "vir"], norder=False)
hf_energy_equation(H1 + H2)


EHF (After simplify): 3 -> 3
 + 1.0\sum_{i}f_{ii}
 + 0.5\sum_{Pij}L_{Pii}L_{Pjj}
 - 0.5\sum_{Pij}L_{Pij}L_{Pji}


In [89]:
H1 = one_e("f", ["occ", "vir"], norder=True)
H2 = DensityFittingERI("L", ["occ", "vir"], norder=True)
ccsd_t1_equation(H1 + H2)


R1 (After simplify): 129 -> 29
 + 1.0f_{ai}
 - 1.0\sum_{j}f_{ji}t_{aj}
 + 1.0\sum_{b}f_{ab}t_{bi}
 + 1.0\sum_{jb}f_{jb}t_{abij}
 - 1.0\sum_{jb}f_{jb}t_{aj}t_{bi}
 - 0.5\sum_{Pjb}L_{Pji}L_{Pab}t_{bj}
 + 0.5\sum_{Pjb}L_{Pjb}L_{Pai}t_{bj}
 + 0.5\sum_{Pjb}L_{Pai}L_{Pjb}t_{bj}
 - 0.5\sum_{Pbj}L_{Pab}L_{Pji}t_{bj}
 - 0.5\sum_{Pjkb}L_{Pji}L_{Pkb}t_{abjk}
 + 0.5\sum_{Pjbk}L_{Pjb}L_{Pki}t_{abjk}
 - 0.5\sum_{Pjbc}L_{Pjb}L_{Pac}t_{bcij}
 + 0.5\sum_{Pbjc}L_{Pab}L_{Pjc}t_{bcij}
 - 0.5\sum_{Pjkb}L_{Pji}L_{Pkb}t_{aj}t_{bk}
 + 0.5\sum_{Pjkb}L_{Pji}L_{Pkb}t_{ak}t_{bj}
 + 0.5\sum_{Pjbk}L_{Pjb}L_{Pki}t_{aj}t_{bk}
 - 0.5\sum_{Pjbk}L_{Pjb}L_{Pki}t_{ak}t_{bj}
 - 0.5\sum_{Pjbc}L_{Pjb}L_{Pac}t_{bi}t_{cj}
 + 0.5\sum_{Pjbc}L_{Pjb}L_{Pac}t_{bj}t_{ci}
 + 0.5\sum_{Pbjc}L_{Pab}L_{Pjc}t_{bi}t_{cj}
 - 0.5\sum_{Pbjc}L_{Pab}L_{Pjc}t_{bj}t_{ci}
 - 0.5\sum_{Pjbkc}L_{Pjb}L_{Pkc}t_{aj}t_{bcik}
 + 0.5\sum_{Pjbkc}L_{Pjb}L_{Pkc}t_{ak}t_{bcij}
 - 0.5\sum_{Pjbkc}L_{Pjb}L_{Pkc}t_{bi}t_{acjk}
 + 0.5\sum_{Pjbkc}L_{Pjb}L_{Pkc}t_{b

In [90]:
def TensorHyperContractionERI(name, spaces, norder=False):
    terms = []
    sym = TensorSym([(0, 1), (1, 0)], [1, 1])
    z = name[0]
    x = name[1]

    for ss in product(spaces, repeat=4):
        # Count how many times each space appears
        inds = [Idx(sum(s == si for s in ss[:i]), si) for i, si in enumerate(ss)]
        p, q, r, s = inds
        P = Idx(0, "aux")
        Q = Idx(1, "aux")
        fops = [FOperator(p, 1), FOperator(r, 1), FOperator(s, 0), FOperator(q, 0)]

        sign = 1
        if norder:
            fops, sign = normal_ordered(fops)

        t = Term(0.5,  # The scalar factor
                [Sigma(P), Sigma(Q), Sigma(p), Sigma(q), Sigma(r), Sigma(s)],
                [Tensor([P, p], x + "(0)"), Tensor([P, q], x + "(1)"),
                 Tensor([Q, r], x + "(2)"), Tensor([Q, s], x + "(3)"),
                 Tensor([P, Q], z, sym=sym)], # The tensor
                fops, []
            )
        t.index_key = {
            "occ": "ijklmno",
            "vir": "abcdefg",
            "aux": "PQRSIJKL"
            }

        terms.append(t)

    return Expression(terms)

H1 = one_e("f", ["occ", "vir"], norder=False)
H2 = TensorHyperContractionERI("ZX", ["occ", "vir"], norder=False)
hf_energy_equation(H1 + H2)


EHF (After simplify): 3 -> 3
 + 1.0\sum_{i}f_{ii}
 + 0.5\sum_{PQij}X_{Pi}X_{Pi}X_{Qj}X_{Qj}Z_{PQ}
 - 0.5\sum_{PQij}X_{Pi}X_{Pj}X_{Qj}X_{Qi}Z_{PQ}


In [91]:
H1 = one_e("f", ["occ", "vir"], norder=True)
H2 = TensorHyperContractionERI("ZX", ["occ", "vir"], norder=True)
ccsd_t1_equation(H1 + H2)


R1 (After simplify): 129 -> 29
 + 1.0f_{ai}
 - 1.0\sum_{j}f_{ji}t_{aj}
 + 1.0\sum_{b}f_{ab}t_{bi}
 + 1.0\sum_{jb}f_{jb}t_{abij}
 - 1.0\sum_{jb}f_{jb}t_{aj}t_{bi}
 + 0.5\sum_{PQjb}X_{Pj}X_{Pi}X_{Qa}X_{Qb}Z_{PQ}t_{bj}
 + 0.5\sum_{PQjb}X_{Pj}X_{Pb}X_{Qa}X_{Qi}Z_{PQ}t_{bj}
 + 0.5\sum_{PQjb}X_{Pa}X_{Pi}X_{Qj}X_{Qb}Z_{PQ}t_{bj}
 + 0.5\sum_{PQbj}X_{Pa}X_{Pb}X_{Qj}X_{Qi}Z_{PQ}t_{bj}
 + 0.5\sum_{PQjkb}X_{Pj}X_{Pi}X_{Qk}X_{Qb}Z_{PQ}t_{abjk}
 + 0.5\sum_{PQjbk}X_{Pj}X_{Pb}X_{Qk}X_{Qi}Z_{PQ}t_{abjk}
 + 0.5\sum_{PQjbc}X_{Pj}X_{Pb}X_{Qa}X_{Qc}Z_{PQ}t_{bcij}
 + 0.5\sum_{PQbjc}X_{Pa}X_{Pb}X_{Qj}X_{Qc}Z_{PQ}t_{bcij}
 + 0.5\sum_{PQjkb}X_{Pj}X_{Pi}X_{Qk}X_{Qb}Z_{PQ}t_{aj}t_{bk}
 - 0.5\sum_{PQjkb}X_{Pj}X_{Pi}X_{Qk}X_{Qb}Z_{PQ}t_{ak}t_{bj}
 + 0.5\sum_{PQjbk}X_{Pj}X_{Pb}X_{Qk}X_{Qi}Z_{PQ}t_{aj}t_{bk}
 - 0.5\sum_{PQjbk}X_{Pj}X_{Pb}X_{Qk}X_{Qi}Z_{PQ}t_{ak}t_{bj}
 + 0.5\sum_{PQjbc}X_{Pj}X_{Pb}X_{Qa}X_{Qc}Z_{PQ}t_{bi}t_{cj}
 - 0.5\sum_{PQjbc}X_{Pj}X_{Pb}X_{Qa}X_{Qc}Z_{PQ}t_{bj}t_{ci}
 + 0.5\sum_{PQbjc}X_{Pa}X_{